***

*Course:* [Math 535](https://people.math.wisc.edu/~roch/mmids/) - Mathematical Methods in Data Science (MMiDS)  
*Chapter:* 8-Neural networks, backpropagation and stochastic gradient descent   
*Author:* [Sebastien Roch](https://people.math.wisc.edu/~roch/), Department of Mathematics, University of Wisconsin-Madison  
*Updated:* July 16, 2024   
*Copyright:* &copy; 2024 Sebastien Roch

***

In [ ]:
# You will need the files:
#     * mmids.py
#     * SAHeart.csv 
# from https://github.com/MMiDS-textbook/MMiDS-textbook.github.io/tree/main/utils
#
# IF RUNNING ON GOOGLE COLAB (RECOMMENDED):
# "Upload to session storage" from the Files tab on the left
# Alternative instructions: https://colab.research.google.com/notebooks/io.ipynb

In [ ]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import torch
import mmids

$\newcommand{\bmu}{\boldsymbol{\mu}}$ 
$\newcommand{\bSigma}{\boldsymbol{\Sigma}}$
$\newcommand{\bfbeta}{\boldsymbol{\beta}}$ 
$\newcommand{\bflambda}{\boldsymbol{\lambda}}$
$\newcommand{\bgamma}{\boldsymbol{\gamma}}$ 
$\newcommand{\bsigma}{{\boldsymbol{\sigma}}}$
$\newcommand{\bpi}{\boldsymbol{\pi}}$
$\newcommand{\btheta}{{\boldsymbol{\theta}}}$ 
$\newcommand{\bphi}{\boldsymbol{\phi}}$ 
$\newcommand{\balpha}{\boldsymbol{\alpha}}$
$\newcommand{\blambda}{\boldsymbol{\lambda}}$
$\renewcommand{\P}{\mathbb{P}}$
$\newcommand{\E}{\mathbb{E}}$
$\newcommand{\indep}{\perp\!\!\!\perp} \newcommand{\bx}{\mathbf{x}}$ 
$\newcommand{\bp}{\mathbf{p}}$
$\renewcommand{\bx}{\mathbf{x}}$
$\newcommand{\bX}{\mathbf{X}}$ 
$\newcommand{\by}{\mathbf{y}}$ 
$\newcommand{\bY}{\mathbf{Y}}$ 
$\newcommand{\bz}{\mathbf{z}}$
$\newcommand{\bZ}{\mathbf{Z}}$ 
$\newcommand{\bw}{\mathbf{w}}$ 
$\newcommand{\bW}{\mathbf{W}}$
$\newcommand{\bv}{\mathbf{v}}$ 
$\newcommand{\bV}{\mathbf{V}}$
$\newcommand{\bfg}{\mathbf{g}}$ 
$\newcommand{\bfh}{\mathbf{h}}$ 
$\newcommand{\horz}{\rule[.5ex]{2.5ex}{0.5pt}}$
$\renewcommand{\S}{\mathcal{S}}$ 
$\newcommand{\X}{\mathcal{X}}$ 
$\newcommand{\var}{\mathrm{Var}}$ 
$\newcommand{\pa}{\mathrm{pa}}$
$\newcommand{\Z}{\mathcal{Z}}$ 
$\newcommand{\bh}{\mathbf{h}}$ 
$\newcommand{\bb}{\mathbf{b}}$ 
$\newcommand{\bc}{\mathbf{c}}$ 
$\newcommand{\cE}{\mathcal{E}}$ 
$\newcommand{\cP}{\mathcal{P}}$
$\newcommand{\bbeta}{\boldsymbol{\beta}}$
$\newcommand{\bLambda}{\boldsymbol{\Lambda}}$
$\newcommand{\cov}{\mathrm{Cov}}$
$\newcommand{\bfk}{\mathbf{k}}$
$\newcommand{\idx}[1]{}$
$\newcommand{\xdi}{}$

## Motivating example:  classifying natural images

In this chapter, we return to the classification problem. This time we consider more complex datasets involving natural images. We have seen an example previously, the MNIST dataset. We use a related dataset known as Fashion-MNIST developed by the [Zalando Research](https://engineering.zalando.com/tags/zalando-research.html). Quoting from their [GitHub repository](https://github.com/zalandoresearch/fashion-mnist):

> Fashion-MNIST is a dataset of Zalando's article images -- consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits.

We first load the data and convert it to an appropriate matrix representation. The data can be accessed with [`torchvision.datasets.FashionMNIST`](https://pytorch.org/vision/stable/generated/torchvision.datasets.FashionMNIST.html).

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset

fashion_mnist = datasets.FashionMNIST(root='./data', train=True, 
                                      download=True, transform=transforms.ToTensor())

For example, the first image and its label are the following. The [`squeeze()`](https://pytorch.org/docs/stable/generated/torch.Tensor.squeeze.html) below removes the color dimension in the image, which is grayscale.

In [ ]:
img, label = fashion_mnist[0]
plt.figure()
plt.imshow(img.squeeze(), cmap='gray')
plt.show()

In [ ]:
label

In [ ]:
def FashionMNIST_get_class_name(label):

    class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", 
    "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

    return class_names[label]

print(f"{label}: '{FashionMNIST_get_class_name(label)}'")

The purpose of this chapter is to develop some of the mathematical tools used to solve this kind of classification problem:

- neural networks,
- backpropagation,
- stochastic gradient descent.

## Background: Jacobian, chain rule, and a brief introduction to automatic differentiation

**Automatic differentiation in PyTorch** We will use [PyTorch](https://pytorch.org/tutorials/). It uses [tensors](https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html)$\idx{tensor}\xdi$, which in many ways behave similarly to Numpy arrays. See [here](https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html) for a quick introduction. We first initialize the tensors. Here each tensor corresponds to a single real variable. With the option [`requires_grad=True`](https://pytorch.org/docs/stable/generated/torch.Tensor.requires_grad.html#torch.Tensor.requires_grad), we indicate that these are variables with respect to which a gradient will be taken later. We initialize the tensors at the values where the derivatives will be computed. If derivatives need to be computed at different values, we need to repeat this process. The function [`.backward()`](https://pytorch.org/docs/stable/generated/torch.Tensor.backward.html) computes the gradient using backpropagation, to which we will return later. The partial derivatives are accessed with [`.grad`](https://pytorch.org/docs/stable/generated/torch.Tensor.grad.html). 

**NUMERICAL CORNER:** This is better understood through an example.

In [ ]:
x1 = torch.tensor(1.0, requires_grad=True)
x2 = torch.tensor(2.0, requires_grad=True)

We define the function. Note that we use
[`torch.exp`](https://pytorch.org/docs/stable/generated/torch.exp.html), the PyTorch implementation of the (element-wise) exponential function. Moreover, as in NumPy, PyTorch allows the use of `**` for [taking a power](https://pytorch.org/docs/stable/generated/torch.pow.html). [Here](https://pytorch.org/docs/stable/name_inference.html) is a list of operations on tensors in PyTorch. 

In [ ]:
f = 3 * (x1 ** 2) + x2 + torch.exp(x1 * x2)

f.backward()

print(x1.grad)  # df/dx
print(x2.grad)  # df/dy

The input parameters can also be vectors, which allows to consider functions of large numbers of variables. Here we use [`torch.sum`](https://pytorch.org/docs/stable/generated/torch.sum.html#torch.sum) for taking a sum of the arguments.

In [ ]:
z = torch.tensor([1., 2., 3.], requires_grad=True)

g = torch.sum(z ** 2)
g.backward()

print(z.grad)  # gradient is (2 z_1, 2 z_2, 2 z_3)

Here is another typical example in a data science context.

In [ ]:
X = torch.randn(3, 2)  # Random dataset (features)
y = torch.tensor([[1., 0., 1.]])  # Dataset (labels)
theta = torch.ones(2, 1, requires_grad=True)  # Parameter assignment

predict = X @ theta  # Classifier with parameter vector theta
loss = torch.sum((predict - y)**2)  # Loss function
loss.backward()  # Compute gradients

print(theta.grad)  # gradient of loss

**CHAT & LEARN** Ask your favorite AI chatbot to explain how to compute a second derivative using PyTorch (it's bit tricky). Ask for code that you can apply to the previous examples. ([Open In Colab](https://colab.research.google.com/github/MMiDS-textbook/MMiDS-textbook.github.io/blob/main/just_the_code/roch_mmids_chap_nn_notebook.ipynb)) $\ddagger$

$\unlhd$

**Implementing gradient descent in PyTorch** Rather than explicitly specifying the gradient function, we could use PyTorch to compute it automatically. This is done next. Note that the descent update is done within [`with torch.no_grad()`](https://pytorch.org/docs/stable/generated/torch.no_grad.html), which ensures that the update operation itself is not tracked for gradient computation. Here the input `x0` as well as the output `xk.numpy(force=True)` are Numpy arrays. The function [`torch.Tensor.numpy()`](https://pytorch.org/docs/stable/generated/torch.Tensor.numpy.html) converts a PyTorch tensor to a Numpy array (see the documentation for an explanation of the `force=True` option). Also, quoting ChatGPT:

> In the given code, `.item()` is used to extract the scalar value from a tensor. In PyTorch, when you perform operations on tensors, you get back tensors as results, even if the result is a single scalar value. `.item()` is used to extract this scalar value from the tensor.


In [ ]:
def gd_with_ad(f, x0, alpha=1e-3, niters=int(1e6)):
    xk = torch.tensor(x0, requires_grad=True, dtype=torch.float)
    
    for _ in range(niters):
        value = f(xk)
        value.backward()

        with torch.no_grad():  
            xk -= alpha * xk.grad

        xk.grad.zero_()

    return xk.numpy(force=True), f(xk).item()

**NUMERICAL CORNER:** We revisit a previous example.

In [ ]:
def f(x):
    return x**3

print(gd_with_ad(f, 2, niters=int(1e4)))

In [ ]:
print(gd_with_ad(f, -2, niters=100))

$\unlhd$

## Building blocks of AI 1: backpropagation

**NUMERICAL CORNER:** To make things more concrete, we consider a specific example. We will use [`torch.linalg.vector_norm`](https://pytorch.org/docs/stable/generated/torch.linalg.vector_norm.html) to compute the Euclidean norm in PyTorch. Suppose $d=3$, $L=1$, $n_1 = 2$, and $K = 2$ with the following choices: 

In [ ]:
x = torch.tensor([1.,0.,-1.], requires_grad=True)
y = torch.tensor([0.,1.])
W0 = torch.tensor([[0.,1.,-1.],[2.,0.,1.]])
W1 = torch.tensor([[-1.,0.],[2.,-1.]])

z0 = x
z1 = W0 @ z0
z2 = W1 @ z1
f = 0.5 * (torch.linalg.vector_norm(y-z2) ** 2)

print(z0)

In [ ]:
print(z1)

In [ ]:
print(z2)

In [ ]:
print(f)

$\unlhd$

**NUMERICAL CORNER:** We return to our concrete example. Using `.T` to convert a column vector into a row vector throws an error in PyTorch, as it is meant to be used only on 2D tensors. Instead, one can use [`torch.unsqueeze`](https://pytorch.org/docs/stable/generated/torch.unsqueeze.html). Below, `(z2 - y).unsqueeze(0)` adds a dimension to `z2 - y`, making it a 2D tensor with shape $(1, 2)$.

In [ ]:
with torch.no_grad():
    F0 = W0
    F1 = W1 @ F0
    grad_f = (z2 - y).unsqueeze(0) @ F1
    
print(F0)

In [ ]:
print(F1)

In [ ]:
print(grad_f)

We can check that we get the same outcome using AD.

In [ ]:
f.backward()
print(x.grad)

$\unlhd$

**NUMERICAL CORNER:** We try our specific example.

In [ ]:
with torch.no_grad():
    G2 = (z2 - y).unsqueeze(0)
    G1 = G2 @ W1
    grad_f = G1 @ W0
    
print(G2)

In [ ]:
print(G1)

In [ ]:
print(grad_f)

We indeed obtain the same answer yet again.

$\unlhd$

**NUMERICAL CORNER:** We return to the concrete example from the previous subsection. This time the matrices `W0` and `W1` require partial derivatives.

In [ ]:
x = torch.tensor([1.,0.,-1.])
y = torch.tensor([0.,1.])
W0 = torch.tensor([[0.,1.,-1.],[2.,0.,1.]], requires_grad=True)
W1 = torch.tensor([[-1.,0.],[2.,-1.]], requires_grad=True)

z0 = x
z1 = W0 @ z0
z2 = W1 @ z1
f = 0.5 * (torch.linalg.vector_norm(y-z2) ** 2)

print(z0)

In [ ]:
print(z1)

In [ ]:
print(z2)

In [ ]:
print(f)

We compute the gradient $\nabla f(\mathbf{w})$ using AD.

In [ ]:
f.backward()

In [ ]:
print(W0.grad)

In [ ]:
print(W1.grad)

These are written in the form of matrix derivatives

$$
\frac{\partial f}{\partial \mathcal{W}_0}
= \begin{pmatrix}
\frac{\partial f}{\partial w_0} &
\frac{\partial f}{\partial w_1} &
\frac{\partial f}{\partial w_2} \\
\frac{\partial f}{\partial w_3} &
\frac{\partial f}{\partial w_4} &
\frac{\partial f}{\partial w_5}
\end{pmatrix}
\quad\text{and}\quad
\frac{\partial f}{\partial \mathcal{W}_1}
= \begin{pmatrix}
\frac{\partial f}{\partial w_6} &
\frac{\partial f}{\partial w_7} \\
\frac{\partial f}{\partial w_8} &
\frac{\partial f}{\partial w_9}
\end{pmatrix}.
$$

We use our formulas to confirm that they match these results. We need the Kronecker product, which in PyTorch is implemented as [`torch.kron`](https://pytorch.org/docs/stable/generated/torch.kron.html).

In [ ]:
with torch.no_grad():
    grad_W0 = torch.kron((z2 - y).unsqueeze(0) @ W1, z0.unsqueeze(0))
    grad_W1 = torch.kron((z2 - y).unsqueeze(0), z1.unsqueeze(0))

print(grad_W0)

In [ ]:
print(grad_W1)

Observe that this time these results are written in vectorized form (i.e., obtained by concatenating the rows). But they do match with the AD output.

$\unlhd$

## Building blocks of AI 2: stochastic gradient descent

For the mini-batch version of SGD, we pick a random sub-sample $\mathcal{B}_t \subseteq \{1,\ldots,n\}$ of size $B$ and take the step

$$
\mathbf{x}^{t+1}
= \mathbf{x}^{t} +\beta \frac{1}{B} \sum_{i\in \mathcal{B}_t} (
b_i - \sigma(\boldsymbol{\alpha}_i^T \mathbf{x}^t) 
) \,\boldsymbol{\alpha}_i.
$$

We modify our previous code for logistic regression. The only change is to pick a random mini-batch which can be fed to the descent update sub-routine as dataset.

In [ ]:
def sigmoid(z): 
    return 1/(1+np.exp(-z))

def pred_fn(x, A): 
    return sigmoid(A @ x)

def loss_fn(x, A, b): 
    return np.mean(-b*np.log(pred_fn(x, A)) - (1 - b)*np.log(1 - pred_fn(x, A)))

def grad_fn(x, A, b):
    return -A.T @ (b - pred_fn(x, A))/len(b)

def desc_update_for_logreg(grad_fn, A, b, curr_x, beta):
    gradient = grad_fn(curr_x, A, b)
    return curr_x - beta*gradient

def sgd_for_logreg(rng, loss_fn, grad_fn, A, b, 
                   init_x, beta=1e-3, niters=int(1e5), batch=40):
    
    curr_x = init_x
    nsamples = len(b)
    for _ in range(niters):
        I = rng.integers(nsamples, size=batch)
        curr_x = desc_update_for_logreg(
            grad_fn, A[I,:], b[I], curr_x, beta)
    
    return curr_x

**NUMERICAL CORNER:** We analyze a dataset from [[ESL](https://web.stanford.edu/~hastie/ElemStatLearn/)], which can be downloaded [here](https://web.stanford.edu/~hastie/ElemStatLearn/data.html). Quoting [[ESL](https://web.stanford.edu/~hastie/ElemStatLearn/), Section 4.4.2] 

> The data [...] are a subset of the Coronary Risk-Factor Study (CORIS) baseline survey, carried out in three rural areas of the Western Cape, South Africa (Rousseauw et al., 1983). The aim of the study was to establish the intensity of ischemic heart disease risk factors in that high-incidence region. The data represent white males between 15 and 64, and the response variable is the presence or absence of myocardial infarction (MI) at the time of the survey (the overall prevalence of MI was 5.1% in this region). There are 160 cases in our data set, and a sample of 302 controls. These data are described in more detail in Hastie and Tibshirani (1987).

We load the data, which we slightly reformatted and look at a summary. 

In [ ]:
data = pd.read_csv('SAHeart.csv')
data.head()

Our goal to predict `chd`, which stands for coronary heart disease, based on the other variables (which are briefly described [here](https://web.stanford.edu/~hastie/ElemStatLearn/datasets/SAheart.info.txt)). We use logistic regression again. 

We first construct the data matrices. We only use three of the predictors.

In [ ]:
feature = data[['tobacco', 'ldl', 'age']].to_numpy()
print(feature)

In [ ]:
label = data['chd'].to_numpy()
A = np.concatenate((np.ones((len(label),1)),feature),axis=1)
b = label

We try mini-batch SGD. 

In [ ]:
seed = 535
rng = np.random.default_rng(seed)
init_x = np.zeros(A.shape[1])
best_x = sgd_for_logreg(rng, loss_fn, grad_fn, A, b, init_x, beta=1e-3, niters=int(1e6))
print(best_x)

The outcome is harder to vizualize. To get a sense of how accurate the result is, we compare our predictions to the true labels. By prediction, let us say that we mean that we predict label $1$ whenever $\sigma(\boldsymbol{\alpha}^T \mathbf{x}) > 1/2$. We try this on the training set. (A better approach would be to split the data into training and testing sets, but we will not do this here.)

In [ ]:
def logis_acc(x, A, b):
    return np.sum((pred_fn(x, A) > 0.5) == b)/len(b)

In [ ]:
logis_acc(best_x, A, b)

$\unlhd$

**NUMERICAL CORNER:** We will use the Fashion MNIST dataset. This example is inspired by [these](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html) [tutorials](https://www.tensorflow.org/tutorials/keras/classification). We first check for the availability of GPUs and load the data.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() 
                      else ("mps" if torch.backends.mps.is_available() 
                            else "cpu"))
print("Using device:", device)

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

seed = 42
torch.manual_seed(seed)

if device.type == 'cuda': # device-specific seeding and settings
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # for multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
elif device.type == 'mps':
    torch.mps.manual_seed(seed)  # MPS-specific seeding

g = torch.Generator()
g.manual_seed(seed)

train_dataset = datasets.FashionMNIST(root='./data', train=True, 
                               download=True, transform=transforms.ToTensor())
test_dataset = datasets.FashionMNIST(root='./data', train=False, 
                              download=True, transform=transforms.ToTensor())

BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, generator=g)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

We used [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), which provides utilities to load the data in batches for training. We took mini-batches of size `BATCH_SIZE = 32` and we apply a random permutation of the samples on every pass over the training data (with the option `shuffle=True`). The function [`torch.manual_seed()`](https://pytorch.org/docs/stable/generated/torch.manual_seed.html) is used to set the global seed for PyTorch operations (e.g., weight initialization). The shuffling in `DataLoader` uses its own separate random number generator, which we initialize with [`torch.Generator()`](https://pytorch.org/docs/stable/generated/torch.Generator.html#torch.Generator) and [`manual_seed()`](https://pytorch.org/docs/stable/generated/torch.Generator.html#torch.Generator.manual_seed). (You can tell from the fact that `seed=42` that Claude explained that one to me...)

**CHAT & LEARN** Ask your favorite AI chatbot to explain the lines:

```
if device.type == 'cuda': # device-specific seeding and settings
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # for multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
elif device.type == 'mps':
    torch.mps.manual_seed(seed)  # MPS-specific seeding
```

$\ddagger$

We implement multinomial logistic regression to learn a classifier for the MNIST data. In PyTorch, composition of functions can be achieved with [`torch.nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html). Our model is:

In [ ]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28 * 28, 10)
).to(device)

The [`torch.nn.Flatten`](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) layer turns each input image into a vector of size $784$ (where $784 = 28^2$ is the number of pixels in each image). After the flattening, we have an affine map from $\mathbb{R}^{784}$ to $\mathbb{R}^{10}$. Note that there is no need to pre-process the inputs by adding $1$s. A constant term (or "bias variable") is automatically added by PyTorch (unless one chooses the option [`bias=False`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)). The final output is $10$-dimensional.

Finally, we are ready to run an optimization method of our choice on the loss function, which are specified next. There are many [optimizers](https://pytorch.org/docs/stable/optim.html#algorithms) available. (See this [post](https://hackernoon.com/demystifying-different-variants-of-gradient-descent-optimization-algorithm-19ae9ba2e9bc) for a brief explanation of many common optimizers.) Here we use SGD as the optimizer. A quick tutorial is [here](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html). The loss function is the [cross-entropy](https://en.wikipedia.org/wiki/Cross_entropy), as implemented by [`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), which first takes the softmax and expects the labels to be class names rather than their one-hot encoding.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3)

We implement special functions for training.

In [ ]:
def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)    
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def training_loop(train_loader, model, loss_fn, optimizer, device, epochs=3):
    for epoch in range(epochs):
        train(train_loader, model, loss_fn, optimizer, device)
        print(f"Epoch {epoch+1}/{epochs}")

An epoch is one training iteration where all samples are iterated once (in a randomly shuffled order). In the interest of time, we train for 10 epochs only. But it does better if you train it longer (try it!). On each pass, we compute the output of the current model, use `backward()` to obtain the gradient, and then perform a descent update with `step()`. We also have to reset the gradients first (otherwise they add up by default). 

In [ ]:
training_loop(train_loader, model, loss_fn, optimizer, device, epochs=10)

Because of the issue of [overfitting](https://en.wikipedia.org/wiki/Overfitting), we use the *test* images to assess the performance of the final classifier. 

In [ ]:
def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    correct = 0    
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            correct += (pred.argmax(dim=1) == y).type(torch.float).sum().item()

    print(f"Test error: {(100*(correct / size)):>0.1f}% accuracy")

In [ ]:
test(test_loader, model, loss_fn, device)

To make a prediction, we take a [`torch.nn.functional.softmax`](https://pytorch.org/docs/stable/generated/torch.nn.functional.softmax.html) of the output of our model. Recall that it is implicitly included in `torch.nn.CrossEntropyLoss`, but is not actually part of `model`. (Note that the softmax itself has no parameter.) 

As an illustration, we do this for each test image. We use [`torch.cat`](https://pytorch.org/docs/stable/generated/torch.cat.html) to concatenate a sequence of tensors into a single tensor.

In [ ]:
import torch.nn.functional as F

def predict_softmax(dataloader, model, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    predictions = []
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            probabilities = F.softmax(pred, dim=1)
            predictions.append(probabilities.cpu())
            
    return torch.cat(predictions, dim=0)

predictions = predict_softmax(test_loader, model, device).numpy()

The result for the first test image is shown below. To make a prediction, we choose the label with the highest probability.

In [ ]:
print(predictions[0])

In [ ]:
predictions[0].argmax(0)

The truth is:

In [ ]:
images, labels = next(iter(test_loader))
images = images.squeeze().numpy()
labels = labels.numpy()

print(f"{labels[0]}: '{mmids.FashionMNIST_get_class_name(labels[0])}'")

Above, `next(iter(test_loader))` loads the first batch of test images. (See [here](https://docs.python.org/3/tutorial/classes.html#iterators) for background on iterators in Python.)

$\unlhd$

## Building blocks of AI 3: neural networks

**NUMERICAL CORNER:** We return to the concrete example from the previous section. We re-write the gradient as

\begin{align*}
\nabla f(\mathbf{w})^T
&= \begin{pmatrix}
[(\mathbf{z}_2 - \mathbf{y})^T
\mathcal{W}_{1} \mathrm{diag}(\mathbf{z}_1 \odot (\mathbf{1} - \mathbf{z}_1))] \otimes \mathbf{z}_0^T &
(\mathbf{z}_2 - \mathbf{y})^T
\otimes \mathbf{z}_1^T
\end{pmatrix}.
\end{align*}

We will use [`torch.nn.functional.sigmoid`](https://pytorch.org/docs/stable/generated/torch.nn.functional.sigmoid.html) and
[`torch.nn.functional.softmax`](https://pytorch.org/docs/stable/generated/torch.nn.functional.softmax.html) for the sigmoid and softmax functions respectively. We also use [`torch.dot`](https://pytorch.org/docs/stable/generated/torch.dot.html) for the inner product (i.e., dot product) of two vectors (as tensors) and [`torch.diag`](https://pytorch.org/docs/stable/generated/torch.diag.html) for the creation of a diagonal matrix with specified entries on its diagonal. 

In [ ]:
import torch.nn.functional as F

x = torch.tensor([1.,0.,-1.])
y = torch.tensor([0.,1.])
W0 = torch.tensor([[0.,1.,-1.],[2.,0.,1.]], requires_grad=True)
W1 = torch.tensor([[-1.,0.],[2.,-1.]], requires_grad=True)

z0 = x
z1 = F.sigmoid(W0 @ z0)
z2 = F.softmax(W1 @ z1, dim=0)
f = -torch.dot(torch.log(z2), y)

print(z0)

In [ ]:
print(z1)

In [ ]:
print(z2)

In [ ]:
print(f)

We compute the gradient $\nabla f(\mathbf{w})$ using AD.

In [ ]:
f.backward()
print(W0.grad)

In [ ]:
print(W1.grad)

We use our formulas to confirm that they match these results.

In [ ]:
with torch.no_grad():
    grad_W0 = torch.kron((z2 - y).unsqueeze(0) @ W1 @ torch.diag(z1 * (1-z1)), z0.unsqueeze(0))
    grad_W1 = torch.kron((z2 - y).unsqueeze(0), z1.unsqueeze(0))

print(grad_W0)

In [ ]:
print(grad_W1)

The results match with the AD output.

$\unlhd$

**NUMERICAL CORNER:** We implement the training of a neural network in PyTorch. We use the Fashion MNIST dataset again. We first load it again. We also check for the availability of GPUs.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() 
                      else ('mps' if torch.backends.mps.is_available() 
                            else 'cpu'))
print('Using device:', device)

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

seed = 42
torch.manual_seed(seed)

if device.type == 'cuda': # device-specific seeding and settings
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # for multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
elif device.type == 'mps':
    torch.mps.manual_seed(seed)  # MPS-specific seeding

g = torch.Generator()
g.manual_seed(seed)

train_dataset = datasets.FashionMNIST(root='./data', train=True, 
                               download=True, transform=transforms.ToTensor())
test_dataset = datasets.FashionMNIST(root='./data', train=False, 
                              download=True, transform=transforms.ToTensor())

BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, generator=g)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

We construct a two-layer model.

In [ ]:
model = nn.Sequential(
    nn.Flatten(),                      # Flatten the input
    nn.Linear(28 * 28, 32),            # First Linear layer with 32 nodes
    nn.Sigmoid(),                      # Sigmoid activation function
    nn.Linear(32, 10)                  # Second Linear layer with 10 nodes (output layer)
).to(device)

As we did for multinomial logistic regression, we use the Adam optimizer and the cross-entropy loss (which in PyTorch includes the softmax function and expects labels to be class names rather than one-hot encoding).

In [ ]:
loss_fn = nn.CrossEntropyLoss()  
optimizer = optim.SGD(model.parameters(), lr=1e-3)

We train for 10 epochs.

In [ ]:
mmids.training_loop(train_loader, model, loss_fn, optimizer, device, epochs=10)

On the test data, we get:

In [ ]:
mmids.test(test_loader, model, loss_fn, device)

Disappointingly, this is significantly less accurate model than what we obtained using multinomial logistic regression. It turns out that using a different optimizer gives much better results.

In [ ]:
loss_fn = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters())
mmids.training_loop(train_loader, model, loss_fn, optimizer, device)

In [ ]:
mmids.test(test_loader, model, loss_fn, device)

**CHAT & LEARN** The text mentions that there are many optimizers available in PyTorch besides SGD. Ask your favorite AI chatbot to explain and implement a different optimizer, such as Adam, Adagrad or RMSprop, for the linear regression problem. Compare the results with those obtained using SGD. $\ddagger$

$\unlhd$